<a href="https://colab.research.google.com/github/Requenamar3/Azure-Data-Studio-Project/blob/main/Realistic_ShippingCostAnalysis_Synthetic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Azure Data Studio Project: Synthetic Data Generation for Shipping and Distribution**

**Objective**
This project creates a realistic dataset for a fictional e-commerce business that ships perishable goods, like custom cakes, across the U.S. The goal is to simulate a shipping and distribution network to help analyze and improve key areas like shipping costs, delivery efficiency, and carrier performance. With this dataset, the project will identify inefficiencies, explore optimization strategies, and support better decision-making.

**Background**
Shipping perishable products is tricky, and getting them to customers on time while keeping costs down is a constant challenge. This dataset is designed to mimic the real-world complexities of logistics. It helps answer important questions like:

* What factors drive up shipping costs?
* Which carriers or routes work best?
* How do holidays and special events like weddings and graduations impact demand and shipping expenses?

To make the data as realistic as possible, the project incorporates assumptions like seasonal order spikes, varying carrier costs, and differences in delivery reliability. By doing this, it simulates the challenges a business might actually face and provides insights to improve shipping operations, cut costs, and deliver a better experience for customers.


# 1.Project Setup


## 1.1 Install Required Libraries
Install the Faker library for generating realistic synthetic data:

In [ ]:
# Install both pands and  Faker library
!pip install pandas faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.2 MB/s eta 0:00:00


## 1.2 Import Libraries

Library Descriptions:
* datetime.timedelta: Manipulates dates to calculate shipping timelines.
* pandas: Creates and manages tabular data structures (DataFrames).
* faker: Generates realistic synthetic data like ZIP codes, states, and dates.
* random: Adds variability to synthetic data generation.
* numpy: Performs numerical operations like random number generation and scaling.

In [ ]:
# libraries needed
from datetime import datetime, timedelta
import pandas as pd
from faker import Faker
import random
import numpy as np # Import numpy library and assign it the alias 'np'


# Initialize Faker
fake = Faker()

In [ ]:
# Import the necessary modules from IPython to manipulate the display settings
from IPython.display import HTML, display

# Use the display function to inject custom HTML and CSS
# This CSS hides all the code cells in the notebook while keeping the output visible
display(HTML("""
    <style>
        .output_wrapper {overflow: auto;} /* Ensures output is scrollable if it overflows */
        .cell.code_cell {display: none;} /* Hides all code cells in the notebook */
    </style>
"""))


##1.3 Initialize Variables
Set up the number of data points and initialize Faker:


In [ ]:
# Define the number of data points for each table. Adjust as needed
num_orders = 20000  # Total orders across 4 years (average 5,000 per year)
num_shipping_transactions = 20000  # One transaction per order
num_transactions = 20000
num_routes = 500  # Number of unique routes
num_zip_codes = 1000  # Number of unique ZIP codes

##1.4 Define Date Range

In [ ]:
from datetime import datetime, timedelta
import random

# Define a helper function to generate random dates within a specified range
def random_date(start_date, end_date):
    """
    Generates a random date between start_date and end_date.

    Args:
    start_date (datetime): The beginning of the date range.
    end_date (datetime): The end of the date range.

    Returns:
    datetime: A randomly generated date within the given range.
    """
    delta = end_date - start_date  # Calculate the total range in days
    random_days = random.randint(0, delta.days)  # Generate a random number of days within the range
    return start_date + timedelta(days=random_days)  # Add the random number of days to start_date


In [ ]:
# The date range spans from January 1, 2020, to November 1, 2024
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 11, 1)
dates = pd.date_range(start=start_date, end=end_date)

##1.5 Simulate Daily Order Volumes with Seasonality and Variability

In [ ]:
def simulate_order_volume(date):
    """
    Simulates daily order volumes with seasonality and variability.

    Args:
        date (datetime): The date for which the order volume is being simulated.

    Returns:
        int: Simulated number of orders for the given date.
    """
    # Define base volumes by month
    base_volume = {
        1: 300,  # January
        2: 450,  # February (Valentine's Day)
        3: 400,  # March
        4: 420,  # April (Easter)
        5: 500,  # May (Mother's Day)
        6: 550,  # June (Wedding/Graduation season)
        7: 580,  # July (Wedding season)
        8: 540,  # August
        9: 480,  # September
        10: 500,  # October
        11: 550,  # November (Thanksgiving)
        12: 700,  # December (Christmas/New Year)
    }

    # Add holiday/event-specific adjustments
    if date.month == 2 and date.day == 14:  # Valentine's Day
        adjustment = 150
    elif date.month == 4 and (12 <= date.day <= 18):  # Easter
        adjustment = 100
    elif date.month == 5 and date.day == 10:  # Mother's Day
        adjustment = 120
    elif date.month in [6, 7, 8]:  # Wedding/Graduation season
        adjustment = 200
    elif date.month == 11 and date.day >= 24:  # Thanksgiving
        adjustment = 250
    elif date.month == 12:  # Christmas/New Year
        adjustment = 300
    else:
        adjustment = 0

    # Introduce random variability to make the data more realistic
    variability = np.random.normal(0, 50)  # Mean=0, Std Dev=50

    # Calculate the final daily order volume
    return max(0, base_volume.get(date.month, 300) + adjustment + variability)

# Generate initial daily order volumes
daily_order_volumes = [simulate_order_volume(date) for date in dates]

# Adjust daily volumes to ensure the total matches 10,000 orders
total_orders = sum(daily_order_volumes)
scaling_factor = 10000 / total_orders
adjusted_daily_volumes = [int(volume * scaling_factor) for volume in daily_order_volumes]

# Ensure exactly 10,000 orders (adjust for rounding errors)
adjusted_daily_volumes[-1] += 10000 - sum(adjusted_daily_volumes)

#2.Data Generation

##2.1 Orders Table
The **Orders Table** captures details about customer orders. This includes the cost of shipping, the distribution center fulfilling the order, the type of delivery, and the purpose of the order (e.g., holidays or events).

**Purpose**

This table is essential for analyzing demand patterns, cost drivers, and event-based shipping trends. It helps answer:
*   How much does it cost to ship orders for different delivery types?
*   Which events or seasons generate higher demand?

**Key Columns**
*   Order_ID: A unique identifier for each order.
*   Distribution_Center_ID: Links the order to a specific distribution center
*   Delivery_Type & Service_Type: Indicates the mode of delivery (e.g., "Ground" or "Air").
*   Cost: The shipping cost for the order.
*   Event_Type & Holiday_Flag: Provides context for seasonal or event-driven demand.
*   Carrier_Reliability_Score: Placeholder for tracking carrier performance.
*   Carrier: The shipping provider for the order (e.g., "FedEx Ground").

**Insights**

*   Analyze shipping costs by delivery type and carrier to identify cost-saving opportunities.
*   Track seasonal and event-driven order spikes to align resources with demand.
*   Assess carrier reliability for on-time delivery and adjust allocation strategies.
*   Evaluate distribution center performance based on order fulfillment and shipping efficiency.
*   Identify trends in customer preferences for delivery modes (e.g., ground vs. air).

In [ ]:
# Generate Orders Table
orders_data = {
    "Order_ID": [f"ORD{str(i).zfill(5)}" for i in range(1, num_orders + 1)],  # Unique Order IDs
    "Distribution_Center_ID": random.choices(['DC1', 'DC2', 'DC3', 'DC4', 'DC5', 'DC6'], k=num_orders),  # Random DCs
    "Customer_ZIP": [f"{random.randint(10000, 99999)}" for _ in range(num_orders)],  # Random ZIP Codes
    "Delivery_Type": random.choices(["Ground", "Air"], k=num_orders),  # Ground or Air delivery
    "Service_Type": random.choices(
        ["Ground", "Standard Overnight", "Priority Overnight"],
        weights=[0.6, 0.25, 0.15],  # Weighted probabilities for service levels
        k=num_orders
    ),  # Shipping service type
    "Cost": [round(random.uniform(10, 300), 2) for _ in range(num_orders)],  # Random shipping cost
    "Event_Type": random.choices(
        population=["Birthday", "Holiday", "Wedding", "Graduation", "Bridal Shower", "Other"],
        weights=[0.3, 0.4, 0.1, 0.1, 0.05, 0.05],
        k=num_orders
    ),  # Weighted event types
    "Holiday_Flag": [1 if event == "Holiday" else 0 for event in random.choices(
        ["Holiday", "Non-Holiday"], weights=[0.4, 0.6], k=num_orders)],  # Holiday flag based on Event_Type
    "Carrier_Reliability_Score": [None] * num_orders,  # Placeholder for future data
    "Carrier": random.choices(
        population=["UPS", "FedEx", "OnTrac"],
        weights=[0.5, 0.4, 0.1],  # Weighted probabilities for provider selection
        k=num_orders
    )  # Shipping provider
}


In [ ]:
# Generate order_date based on daily adjusted volumes
# Create sequential dates for the entire range, ensuring it matches num_orders
date_list = [dates[i % len(dates)] for i in range(num_orders)]  # Use modulo to cycle through dates if needed
orders_data["order_date"] = date_list  # Assign the generated dates

# Add Daily Order Volumes, ensuring it matches num_orders
# Repeat daily order volumes for all orders based on their order_date
expanded_order_volumes = [adjusted_daily_volumes[i % len(adjusted_daily_volumes)] for i in range(num_orders)]  # Use modulo to cycle through volumes if needed
orders_data["Daily_Order_Volume"] = expanded_order_volumes

# Create the Orders DataFrame
orders_df = pd.DataFrame(orders_data)


In [ ]:
# Generate Promised and Actual Delivery Dates
# Add promised delivery dates based on the order date
orders_df['promised_delivery_date'] = orders_df['order_date'] + pd.to_timedelta(np.random.randint(1, 5, size=len(orders_df)), unit='D')

# Add actual delivery dates with variability (e.g., early or late deliveries)
orders_df['actual_delivery_date'] = orders_df['promised_delivery_date'] + pd.to_timedelta(np.random.randint(-2, 6, size=len(orders_df)), unit='D')

# Calculate Delivery Difference and Status
# Calculate the delivery difference (days early/late)
orders_df['delivery_difference'] = (orders_df['actual_delivery_date'] - orders_df['promised_delivery_date']).dt.days

# Categorize delivery status
def categorize_delivery(diff):
    """
    Categorize delivery status based on the difference in days.

    Args:
        diff (int): The difference between actual and promised delivery dates.

    Returns:
        str: 'Early', 'On Time', or 'Late' based on the difference.
    """
    if diff < 0:
        return 'Early'
    elif diff == 0:
        return 'On Time'
    else:
        return 'Late'

# Apply the categorization to each order
orders_df['delivery_status'] = orders_df['delivery_difference'].apply(categorize_delivery)

# Report and Inspect the Orders Table
# Print a title for the table overview
report_title = "Orders Table Overview: Summary of Customer Orders"
print("\n" + "=" * len(report_title))
print(report_title)
print("=" * len(report_title))

# Display the first few rows to preview the table
print("\nPreview of the first 5 rows:")
display(orders_df.head())

# Show detailed structure and data types
print("\nStructure and Info:")
orders_df.info()

# Print descriptive statistics for numerical columns
print("\nDescriptive Statistics for Numerical Data:")
print(orders_df.describe())



Orders Table Overview: Summary of Customer Orders

Preview of the first 5 rows:


,Order_ID,Distribution_Center_ID,Customer_ZIP,Delivery_Type,Service_Type,Cost,Event_Type,Holiday_Flag,Carrier_Reliability_Score,Carrier,order_date,Daily_Order_Volume,promised_delivery_date,actual_delivery_date,delivery_difference,delivery_status
0,ORD00001,DC4,93730,Ground,Ground,219.63,Birthday,1,None,FedEx,2020-01-01,3,2020-01-03,2020-01-08,5,Late
1,ORD00002,DC6,70204,Air,Ground,161.36,Holiday,0,None,UPS,2020-01-02,2,2020-01-04,2020-01-03,-1,Early
2,ORD00003,DC2,53086,Ground,Ground,10.01,Graduation,0,None,UPS,2020-01-03,2,2020-01-07,2020-01-12,5,Late
3,ORD00004,DC3,33043,Air,Ground,136.02,Holiday,1,None,FedEx,2020-01-04,3,2020-01-05,2020-01-04,-1,Early
4,ORD00005,DC2,79080,Ground,Standard Overnight,215.95,Holiday,1,None,OnTrac,2020-01-05,3,2020-01-09,2020-01-12,3,Late



Structure and Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Order_ID                   20000 non-null  object        
 1   Distribution_Center_ID     20000 non-null  object        
 2   Customer_ZIP               20000 non-null  object        
 3   Delivery_Type              20000 non-null  object        
 4   Service_Type               20000 non-null  object        
 5   Cost                       20000 non-null  float64       
 6   Event_Type                 20000 non-null  object        
 7   Holiday_Flag               20000 non-null  int64         
 8   Carrier_Reliability_Score  0 non-null      object        
 9   Carrier                    20000 non-null  object        
 10  order_date                 20000 non-null  datetime64[ns]
 11  Daily_Order_Volume         20000 non-null  int

In [ ]:
# Calculate delivery difference (days early/late)
orders_df['delivery_difference'] = (orders_df['actual_delivery_date'] - orders_df['promised_delivery_date']).dt.days

# Categorize delivery status
def categorize_delivery(diff):
    """
    Categorize delivery status based on the difference in days.

    Args:
        diff (int): The difference between actual and promised delivery dates.

    Returns:
        str: 'Early', 'On Time', or 'Late' based on the difference.
    """
    if diff < 0:
        return 'Early'
    elif diff == 0:
        return 'On Time'
    else:
        return 'Late'

# Apply the categorization to each order
orders_df['delivery_status'] = orders_df['delivery_difference'].apply(categorize_delivery)

# Preview the updated Orders DataFrame
print("\nUpdated Orders Table with Delivery Information:")
display(orders_df[['order_date', 'promised_delivery_date', 'actual_delivery_date', 'delivery_difference', 'delivery_status']].head())



Updated Orders Table with Delivery Information:


,order_date,promised_delivery_date,actual_delivery_date,delivery_difference,delivery_status
0,2020-01-01,2020-01-03,2020-01-08,5,Late
1,2020-01-02,2020-01-04,2020-01-03,-1,Early
2,2020-01-03,2020-01-07,2020-01-12,5,Late
3,2020-01-04,2020-01-05,2020-01-04,-1,Early
4,2020-01-05,2020-01-09,2020-01-12,3,Late


In [ ]:
# Analyze On-Time Delivery (OTD) Metrics
# Calculate OTD percentage
total_orders = len(orders_df)
on_time_orders = len(orders_df[orders_df['delivery_status'] == 'On Time'])
otd_percentage = (on_time_orders / total_orders) * 100

print(f"On-Time Delivery (OTD) Rate: {otd_percentage:.2f}%")

# Summary by delivery status
delivery_summary = orders_df['delivery_status'].value_counts(normalize=True) * 100
print("\nDelivery Status Summary (%):")
print(delivery_summary)

# Example: Analyze late deliveries by carrier (if carrier column exists)
if 'Distribution_Center_ID' in orders_df.columns:  # Assuming you track carriers or distribution centers
    late_by_dc = orders_df[orders_df['delivery_status'] == 'Late'].groupby('Distribution_Center_ID')['delivery_difference'].mean()
    print("\nAverage Late Delivery Days by Distribution Center:")
    print(late_by_dc)


On-Time Delivery (OTD) Rate: 12.23%

Delivery Status Summary (%):
delivery_status
Late       62.835
Early      24.935
On Time    12.230
Name: proportion, dtype: float64

Average Late Delivery Days by Distribution Center:
Distribution_Center_ID
DC1    2.983989
DC2    3.006348
DC3    2.999040
DC4    3.009528
DC5    3.009551
DC6    3.026738
Name: delivery_difference, dtype: float64


##2.2 Product Table

The Product Table provides details about products in the cake decorating business, supporting inventory, pricing, and seasonal demand analysis.

**Purpose:**

Analyze product attributes (e.g., weight, shelf life, packaging cost).
Understand seasonal trends and product variability.
Support inventory and pricing strategies.

**Key Columns:**

  * Product_ID & Product_Code: Unique product identifiers.
  * Product_Name & Product_Type: Descriptive names and categories (e.g., Cake, Cupcake).
  * Occasion: Relevant events (e.g., Wedding, Holiday).
  * Base_Weight & Dimensions: Physical characteristics.
  * Shelf_Life & Is_Perishable: Storage-related attributes.
  * Packaging_Cost & Category: Cost and classification.
  * Seasonality & Final_Weight: Seasonal demand and adjusted weights.

**Insights:**

Link perishability to inventory turnover.
Optimize packaging and product mix.
Track seasonal product demand for better planning.

In [ ]:
# Generate Product Table for Cake Decorating Business
product_data = {
    "Product_ID": ["P1", "P2", "P3", "P4", "P5", "P6", "P7"],  # Unique Product IDs
    "Product_Code": ["CK001", "CK002", "CC003", "CC004", "DS005", "DS006", "DS007"],  # Unique product codes
    "Product_Name": [
        "Chocolate Wedding Cake", "Vanilla Birthday Cake", "Red Velvet Cupcake",
        "Lemon Cupcake", "Macarons", "Eclairs", "Custom Decorated Cookies"
    ],  # Descriptive product names
    "Product_Type": ["Cake", "Cake", "Cupcake", "Cupcake", "Dessert", "Dessert", "Cookies"],  # General categories
    "Occasion": [
        "Wedding", "Birthday", "Wedding", "All Year", "All Year", "Holiday", "Custom"
    ],  # Suitable occasions for each product
    "Base_Weight": [8.0, 6.0, 1.0, 1.0, 0.5, 0.8, 2.0],  # Base weight in pounds
    "Dimensions": ["12x12x8", "10x10x6", "3x3x3", "3x3x3", "2x2x2", "5x3x2", "6x6x1"],  # Dimensions in inches
    "Shelf_Life": [5, 5, 3, 3, 7, 4, 10],  # Shelf life in days
    "Packaging_Cost": [3.0, 2.5, 0.5, 0.5, 0.7, 1.0, 0.8],  # Packaging cost per unit
    "Category": ["Custom Cake", "Custom Cake", "Cupcake", "Cupcake", "Dessert", "Dessert", "Cookies"],  # Product category
    "Is_Perishable": [1, 1, 1, 1, 0, 1, 0],  # 1 = Yes, 0 = No
    "Seasonality": [
        "All Year", "All Year", "Wedding", "All Year", "All Year", "Holiday", "Custom"
    ],  # When the product is in demand
}

# Convert to DataFrame
products_df = pd.DataFrame(product_data)

# Add slight variability to weights for realism
products_df['Weight_Variability'] = products_df['Base_Weight'].apply(lambda x: round(np.random.uniform(-0.5, 0.5), 2))
products_df['Final_Weight'] = products_df['Base_Weight'] + products_df['Weight_Variability']

# Display the first few rows to preview the Product Table
print("\nPreview of the first 5 rows of the Product Table:")
display(products_df.head())

# Show detailed structure and data types of the Product Table
print("\nStructure and Info of the Product Table:")
products_df.info()

# Print descriptive statistics for numerical columns in the Product Table
print("\nDescriptive Statistics for Numerical Data in the Product Table:")
print(products_df.describe())

# Save Product Table to CSV
products_df.to_csv("CakeDecoratingProducts.csv", index=False)
print("Product table created and saved to 'CakeDecoratingProducts.csv'")




Preview of the first 5 rows of the Product Table:


,Product_ID,Product_Code,Product_Name,Product_Type,Occasion,Base_Weight,Dimensions,Shelf_Life,Packaging_Cost,Category,Is_Perishable,Seasonality,Weight_Variability,Final_Weight
0,P1,CK001,Chocolate Wedding Cake,Cake,Wedding,8.0,12x12x8,5,3.0,Custom Cake,1,All Year,0.43,8.43
1,P2,CK002,Vanilla Birthday Cake,Cake,Birthday,6.0,10x10x6,5,2.5,Custom Cake,1,All Year,-0.27,5.73
2,P3,CC003,Red Velvet Cupcake,Cupcake,Wedding,1.0,3x3x3,3,0.5,Cupcake,1,Wedding,0.14,1.14
3,P4,CC004,Lemon Cupcake,Cupcake,All Year,1.0,3x3x3,3,0.5,Cupcake,1,All Year,0.10,1.10
4,P5,DS005,Macarons,Dessert,All Year,0.5,2x2x2,7,0.7,Dessert,0,All Year,0.05,0.55



Structure and Info of the Product Table:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Product_ID          7 non-null      object 
 1   Product_Code        7 non-null      object 
 2   Product_Name        7 non-null      object 
 3   Product_Type        7 non-null      object 
 4   Occasion            7 non-null      object 
 5   Base_Weight         7 non-null      float64
 6   Dimensions          7 non-null      object 
 7   Shelf_Life          7 non-null      int64  
 8   Packaging_Cost      7 non-null      float64
 9   Category            7 non-null      object 
 10  Is_Perishable       7 non-null      int64  
 11  Seasonality         7 non-null      object 
 12  Weight_Variability  7 non-null      float64
 13  Final_Weight        7 non-null      float64
dtypes: float64(4), int64(2), object(8)
memory usage: 912.0+ bytes

Descr

##2.3 Distribution Centers Table
The Distribution Centers Table outlines the locations and volume allocations of the distribution hubs.

**Purpose**

This table provides insights into the logistics network and volume management across distribution centers. It helps identify:
*   Which centers are handling the most volume?
*   Are the centers' allocations optimal based on geographic demand?

**Key Columns**
*   Center_ID: Unique identifier for each distribution center.
*   Location: The city and state of the center.
*   ZIP_Code: The primary ZIP code of the center.
*   Volume_Allocation: The percentage of total shipments managed by the center.

**Insights**
*   Evaluate the allocation of shipping volumes to different distribution centers to determine if they align with regional demand.
*   Identify underutilized or overburdened centers by comparing volume *   allocation percentages to actual shipping data.
*   Analyze the geographical distribution of centers to ensure optimal coverage and identify potential gaps in the network.
*   Assess if certain centers are handling orders for states far from their location, increasing costs unnecessarily.


In [ ]:
# Generate Distribution Centers Table

# Create a dictionary to hold the distribution center data.
distribution_centers_data = {
    "Center_ID": ['DC1', 'DC2', 'DC3', 'DC4', 'DC5', 'DC6'],
    "Location": ['Los Angeles, CA', 'Miami, FL', 'Dallas, TX', 'Bronx, NY', 'Hatfield, PA', 'Chicago, IL'],
    "ZIP_Code": [fake.zipcode_in_state(state) for state in ['CA', 'FL', 'TX', 'NY', 'PA', 'IL']],
    "Volume_Allocation": [0.20, 0.12, 0.10, 0.26, 0.18, 0.14]  # Based on allocation rules
}

distribution_centers_df = pd.DataFrame(distribution_centers_data)
distribution_centers_df.to_csv("Distribution_Centers.csv", index=False)
print("Distribution Centers table created and saved to 'Distribution_Centers.csv'")

# Convert the dictionary into a pandas DataFrame for easier data manipulation and analysis.
distribution_centers_df = pd.DataFrame(distribution_centers_data)

# Report: Distribution Centers Table Overview
report_title = "Distribution Centers Table Overview: Summary of Distribution Network"
print("\n" + "=" * len(report_title))
print(report_title)
print("=" * len(report_title))

# Display the first few rows of the DataFrame to give an overview of the distribution center data.
print("\nPreview of the first 5 rows:")
display(distribution_centers_df.head())

# Provide a summary of the DataFrame structure, including column names, data types, and non-null counts.
print("\nStructure and Info:")
distribution_centers_df.info()

# Print descriptive statistics for numerical columns, like the volume allocation percentages.
print("\nDescriptive Statistics for Numerical Data:")
print(distribution_centers_df.describe())


Distribution Centers table created and saved to 'Distribution_Centers.csv'

Distribution Centers Table Overview: Summary of Distribution Network

Preview of the first 5 rows:


,Center_ID,Location,ZIP_Code,Volume_Allocation
0,DC1,"Los Angeles, CA",94355,0.20
1,DC2,"Miami, FL",34811,0.12
2,DC3,"Dallas, TX",77526,0.10
3,DC4,"Bronx, NY",11010,0.26
4,DC5,"Hatfield, PA",16557,0.18



Structure and Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Center_ID          6 non-null      object 
 1   Location           6 non-null      object 
 2   ZIP_Code           6 non-null      object 
 3   Volume_Allocation  6 non-null      float64
dtypes: float64(1), object(3)
memory usage: 320.0+ bytes

Descriptive Statistics for Numerical Data:
       Volume_Allocation
count           6.000000
mean            0.166667
std             0.058878
min             0.100000
25%             0.125000
50%             0.160000
75%             0.195000
max             0.260000


##2.4 Carriers Table
The Carriers Table defines the shipping providers and their associated costs and service types.

**Purpose**

This table is crucial for understanding the cost-effectiveness of different carriers. It supports:

*   Identifying the most cost-efficient carriers for specific service types.
*   Analyzing carrier reliability and delivery times.

**Key Columns**

*   **Carrier_ID**: Unique identifier for each carrier.
*   **Name**: The name of the carrier (e.g., "UPS Ground").
*   **Flat_Rate**: The fixed cost per service provided by the carrier.
*   **Service_Type**: Specifies the mode of transport (e.g., "Ground" or "Air").

**Insights**
*   Compare flat rate costs across carriers to determine the most cost-effective options for ground and air services.
*   Evaluate the balance of carrier usage (e.g., are certain carriers handling more shipments than others?).
*   Analyze carrier specialization to match the best carrier with specific service types (e.g., ground vs. air).
*   Track how flat rate costs impact total shipping costs when combined with route distances and delivery types.


In [ ]:
# Generate Carriers Table Data
carriers_data = {
    "Carrier_ID": ["C1", "C2", "C3", "C4", "C5", "C6"],  # Unique carrier IDs
    "Carrier_Name": ["UPS", "FedEx", "OnTrac", "FedEx", "FedEx", "UPS"],  # Carrier providers
    "Service_Type": ["Ground", "Ground", "Ground", "Standard Overnight", "Priority Overnight", "Air"],  # Service levels
    "Flat_Rate": [2.5, 3.0, 2.2, 7.0, 8.5, 6.5],  # Flat shipping rates (example values)
    "On_Time_Rate": [0.88, 0.85, 0.80, 0.90, 0.92, 0.89],  # Probability of on-time deliveries
    "Late_Days": [[1, 2, 3], [2, 3, 4], [3, 4, 5], [1, 2], [1, 2], [2, 3]]  # List of possible late days for each carrier
}

# Convert the dictionary to a DataFrame
carriers_df = pd.DataFrame(carriers_data)

# Save the Carriers Table to a CSV file
carriers_df.to_csv("Carriers.csv", index=False)
print("Carriers table created and saved to 'Carriers.csv'")

# Display the first few rows of the Carriers DataFrame to get a quick look at the data
print("\nPreview of the first 5 rows:")
display(carriers_df.head())

# Show the structure and details of the DataFrame, including data types and non-null counts for each column
print("\nStructure and Info:")
carriers_df.info()

# Print out statistics for numerical columns (e.g., flat rates for shipping services)
print("\nDescriptive Statistics for Numerical Data:")
print(carriers_df.describe())


Carriers table created and saved to 'Carriers.csv'

Preview of the first 5 rows:


,Carrier_ID,Carrier_Name,Service_Type,Flat_Rate,On_Time_Rate,Late_Days
0,C1,UPS,Ground,2.5,0.88,"[1, 2, 3]"
1,C2,FedEx,Ground,3.0,0.85,"[2, 3, 4]"
2,C3,OnTrac,Ground,2.2,0.80,"[3, 4, 5]"
3,C4,FedEx,Standard Overnight,7.0,0.90,"[1, 2]"
4,C5,FedEx,Priority Overnight,8.5,0.92,"[1, 2]"



Structure and Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Carrier_ID    6 non-null      object 
 1   Carrier_Name  6 non-null      object 
 2   Service_Type  6 non-null      object 
 3   Flat_Rate     6 non-null      float64
 4   On_Time_Rate  6 non-null      float64
 5   Late_Days     6 non-null      object 
dtypes: float64(2), object(4)
memory usage: 416.0+ bytes

Descriptive Statistics for Numerical Data:
       Flat_Rate  On_Time_Rate
count   6.000000      6.000000
mean    4.950000      0.873333
std     2.704626      0.042740
min     2.200000      0.800000
25%     2.625000      0.857500
50%     4.750000      0.885000
75%     6.875000      0.897500
max     8.500000      0.920000


##2.5 Routes Details Table
The Routes Details Table captures information about the shipping routes connecting distribution centers to customer locations. It includes geographical details, transport modes, and carrier assignments for each route.

**Purpose**:
This table is essential for analyzing the logistics network and optimizing routes. It helps answer:

*    Which routes are the most cost-effective based on distance and preferred transport mode?
*    How are carriers allocated to specific routes, and how does that impact efficiency?
*    What trends exist in preferred modes of transport for long versus short routes?

**Key Columns**

*    **Route_ID**: A unique identifier for each route.
*    **Center_ID**: Links the route to a specific distribution center.
*    **From_ZIP & From_State**: Represents the starting location of the route (distribution center ZIP code and state).
*    **To_ZIP & To_State**: Represents the destination location (customer ZIP code and state).
*    **Distance**: The distance (in miles) between the distribution center and the customer location.
*    **Preferred_Mode**: Indicates the preferred transport mode ("Ground" or "Air") for the route.
*    **Carrier_ID**: Links the route to a specific carrier for execution.

**Insights**
*    Analyze the cost implications of long-distance versus short-distance routes and the preference for air or ground transport.
*    Identify high-cost routes and opportunities to optimize transport modes (e.g., shifting air to ground for cost savings).
*    Evaluate carrier usage patterns to ensure balanced allocation and avoid over-reliance on specific carriers.
*    Use geographical trends (e.g., frequently serviced states or ZIP codes) to plan for expanding distribution centers or addressing inefficiencies.


In [ ]:
# 4. Generate Routes Details Table
# This table defines the routes between distribution centers and customer ZIP codes, including distances and preferred shipping modes.

# Create a dictionary to hold route details data.

routes_details_data = {
    "Route_ID": [f"R{i}" for i in range(1, 101)],  # Example: Generate 100 routes
    "Center_ID": [random.choice(["DC1", "DC2", "DC3", "DC4", "DC5", "DC6"]) for _ in range(100)],
    "From_ZIP": [fake.zipcode() for _ in range(100)],  # Random ZIP codes for source
    "From_State": [fake.state_abbr() for _ in range(100)],  # Random state abbreviations
    "To_ZIP": [fake.zipcode() for _ in range(100)],  # Random ZIP codes for destination
    "To_State": [fake.state_abbr() for _ in range(100)],  # Random state abbreviations
    "Distance": [round(random.uniform(50, 3000), 2) for _ in range(100)],  # Distance in miles
    "Preferred_Mode": [random.choice(["Ground", "Air"]) for _ in range(100)],  # Preferred shipping mode
    "Carrier_ID": [random.choice(["C1", "C2", "C3", "C4", "C5", "C6"]) for _ in range(100)]  # Carrier IDs
}

routes_details_df = pd.DataFrame(routes_details_data)
routes_details_df.to_csv("Routes_Details.csv", index=False)
print("Routes Details table created and saved to 'Routes_Details.csv'")


# Convert the dictionary into a pandas DataFrame.
routes_details_df = pd.DataFrame(routes_details_data)

# Report: Routes Details Table Overview
report_title = "Routes Details Table Overview: Summary of Distribution Routes"
print("\n" + "=" * len(report_title))
print(report_title)
print("=" * len(report_title))

# Display the first few rows of the Routes Details DataFrame.
print("\nPreview of the first 5 rows:")
display(routes_details_df.head())

# Show the structure and details of the DataFrame.
print("\nStructure and Info:")
routes_details_df.info()

# Print out statistics for numerical columns like distances.
print("\nDescriptive Statistics for Numerical Data:")
print(routes_details_df.describe())


Routes Details table created and saved to 'Routes_Details.csv'

Routes Details Table Overview: Summary of Distribution Routes

Preview of the first 5 rows:


,Route_ID,Center_ID,From_ZIP,From_State,To_ZIP,To_State,Distance,Preferred_Mode,Carrier_ID
0,R1,DC2,93787,IA,25433,GA,1570.30,Air,C4
1,R2,DC4,65426,NM,51388,CA,2962.49,Air,C5
2,R3,DC1,15059,AZ,02621,CA,1432.69,Air,C3
3,R4,DC3,95931,SD,51993,AZ,952.85,Ground,C6
4,R5,DC4,13364,HI,48174,NY,2363.78,Ground,C4



Structure and Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Route_ID        100 non-null    object 
 1   Center_ID       100 non-null    object 
 2   From_ZIP        100 non-null    object 
 3   From_State      100 non-null    object 
 4   To_ZIP          100 non-null    object 
 5   To_State        100 non-null    object 
 6   Distance        100 non-null    float64
 7   Preferred_Mode  100 non-null    object 
 8   Carrier_ID      100 non-null    object 
dtypes: float64(1), object(8)
memory usage: 7.2+ KB

Descriptive Statistics for Numerical Data:
          Distance
count   100.000000
mean   1467.257100
std     788.492056
min      52.390000
25%     882.887500
50%    1525.660000
75%    2067.447500
max    2962.490000


##2.6 Shipping Transactions Table

The Shipping Transactions Table provides a detailed log of all shipping events, including costs, timelines, and carrier assignments. It ties together data from the orders, routes, and carriers.

**Purpose**

This table is critical for tracking and analyzing the performance of the shipping process. It helps answer:

*    How do shipping costs vary based on carrier, route, or delivery type?
*    What percentage of shipments are delivered on time?
*    Are there any seasonal trends in shipping activity and associated costs?

**Key Columns**
*    **Transaction_ID**: A unique identifier for each shipping transaction.
*    **Order_ID**: Links the transaction to a specific order from the Orders Table.
*    **Carrier_ID**: Indicates the carrier responsible for the shipment.
*    **Route_ID**: Links the transaction to a specific route from the Routes Details Table.
*    **Shipping_Cost**: The total cost of the shipping transaction.
*    **Transaction_Date**: The date the transaction was created.
*    **Scheduled_Delivery_Date**: The expected delivery date for the shipment.
*    **Actual_Delivery_Date**: The date the shipment was delivered (assumed to match the scheduled date in this dataset).
*    **Shipping_Date**: The date the shipment was sent, typically 1 to 3 days before the scheduled delivery date.

**Insights**

*    Evaluate on-time delivery performance by comparing scheduled and actual delivery dates.
*    Identify high-cost transactions and trends in shipping costs by carrier, route, or season to enable cost optimization
*    Track spikes in shipping activity and costs during peak periods (holidays or seasonal events) to prepare for increased demand.
*    Assess the impact of shipping date proximity to the transaction date on carrier performance and customer satisfaction.

In [ ]:
# Generate Shipping Transactions Table
# This table records shipping transactions, including costs, carrier selection, and delivery timelines.

# Create a dictionary to hold shipping transaction data
shipping_transactions_data = {
    # Generate a unique Transaction ID for each transaction
    "Transaction_ID": [f"TXN{str(i).zfill(4)}" for i in range(1, num_transactions + 1)],

    # Randomly assign an Order ID from the Orders DataFrame to each transaction
    "Order_ID": random.choices(orders_df["Order_ID"], k=num_transactions),

    # Randomly assign a Carrier ID from the Carriers DataFrame to each transaction
    "Carrier_ID": random.choices(carriers_df["Carrier_ID"], k=num_transactions),

    # Randomly assign a Route ID from the Routes Details DataFrame to each transaction
    # Ensure `routes_details_data` is replaced with a valid DataFrame if it exists
    "Route_ID": [f"RT{str(i).zfill(3)}" for i in range(1, num_transactions + 1)],

    # Generate a random shipping cost between $10 and $300, rounded to 2 decimal places
    "Shipping_Cost": [round(random.uniform(10, 300), 2) for _ in range(num_transactions)],

    # Assign a random transaction date within the specified date range
    "Transaction_Date": [random_date(start_date, end_date) for _ in range(num_transactions)],

    # Generate scheduled delivery dates (1 to 5 days after the transaction date)
    "Scheduled_Delivery_Date": [
        random_date(start_date, end_date) + timedelta(days=random.randint(1, 5))
        for _ in range(num_transactions)
    ],

    # Generate actual delivery dates (on time, 1 day late, or 1 day early)
    "Actual_Delivery_Date": [
        random_date(start_date, end_date) + timedelta(days=random.choice([-1, 0, 1]))
        for _ in range(num_transactions)
    ]
}

# Convert the dictionary to a Pandas DataFrame
shipping_transactions_df = pd.DataFrame(shipping_transactions_data)

# Add Shipping Dates (1 to 3 days before the Scheduled Delivery Date)
shipping_transactions_df['Shipping_Date'] = pd.to_datetime(shipping_transactions_df['Scheduled_Delivery_Date']) - pd.to_timedelta(
    [random.randint(1, 3) for _ in range(num_transactions)], unit='D'
)

# Report: Shipping Transactions Table Overview
report_title = "Shipping Transactions Table Overview: Summary of Shipping Events"
print("\n" + "=" * len(report_title))
print(report_title)
print("=" * len(report_title))

# Display the first few rows of the Shipping Transactions DataFrame
print("\nPreview of the first 5 rows:")
display(shipping_transactions_df.head())

# Show the structure and details of the DataFrame
print("\nStructure and Info:")
shipping_transactions_df.info()

# Print out statistics for numerical columns like shipping costs
print("\nDescriptive Statistics for Numerical Data:")
print(shipping_transactions_df.describe())



Shipping Transactions Table Overview: Summary of Shipping Events

Preview of the first 5 rows:


,Transaction_ID,Order_ID,Carrier_ID,Route_ID,Shipping_Cost,Transaction_Date,Scheduled_Delivery_Date,Actual_Delivery_Date,Shipping_Date
0,TXN0001,ORD03848,C2,RT001,189.15,2020-12-20,2020-11-13,2024-07-11,2020-11-11
1,TXN0002,ORD07354,C5,RT002,276.12,2024-06-02,2020-06-23,2023-04-09,2020-06-20
2,TXN0003,ORD07078,C4,RT003,178.62,2020-08-17,2020-11-16,2020-10-17,2020-11-14
3,TXN0004,ORD06047,C4,RT004,135.56,2024-05-16,2021-07-22,2021-01-17,2021-07-21
4,TXN0005,ORD06033,C6,RT005,85.60,2023-05-06,2022-06-24,2024-05-04,2022-06-23



Structure and Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Transaction_ID           20000 non-null  object        
 1   Order_ID                 20000 non-null  object        
 2   Carrier_ID               20000 non-null  object        
 3   Route_ID                 20000 non-null  object        
 4   Shipping_Cost            20000 non-null  float64       
 5   Transaction_Date         20000 non-null  datetime64[ns]
 6   Scheduled_Delivery_Date  20000 non-null  datetime64[ns]
 7   Actual_Delivery_Date     20000 non-null  datetime64[ns]
 8   Shipping_Date            20000 non-null  datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(4)
memory usage: 1.4+ MB

Descriptive Statistics for Numerical Data:
       Shipping_Cost            Transaction_Date     Scheduled_Delivery_Date  \
count

##2.7 Generate ZIP Codes Table

The ZIP Codes Table provides a mapping of ZIP codes to specific routes in the logistics network. This table complements the Routes Details Table by adding granularity, enabling detailed analysis of service areas and their connection to routes.

**Purpose**

This table is essential for understanding the geographical distribution of routes and optimizing coverage. It helps answer:

*    Which ZIP codes are associated with specific routes?
*    Are there clusters of ZIP codes serviced by the same route, and do they indicate inefficiencies?
*    How can the data be used to plan for expanding or reallocating distribution resources?

**Key Columns**

*    **ZIP_Code:** A unique ZIP code representing a service area.
*    **Route_ID:** Links the ZIP code to a specific route from the Routes Details Table.

**Insights**
*    Visualize the geographical reach of each route and identify underserved areas.
*    Analyze clusters of ZIP codes assigned to the same route to improve efficiency.
*    Combine this table with shipping and order data to track demand patterns by region.
*    Use this data to identify opportunities for reassigning ZIP codes to more cost-effective or efficient routes.



In [ ]:
# Generate ZIP Codes Table Data
# This table maps unique ZIP codes to the routes they are serviced by.

# Create a dictionary to store ZIP code data.

zip_codes_data = {
    "ZIP_Code": [fake.zipcode() for _ in range(num_zip_codes)],
    "Route_ID": [random.choice(routes_details_data["Route_ID"]) for _ in range(num_zip_codes)]
}

zip_codes_df = pd.DataFrame(zip_codes_data)
zip_codes_df.to_csv("Zip_Codes.csv", index=False)
print("Zip Codes table created and saved to 'Zip_Codes.csv'")


# Convert the dictionary into a pandas DataFrame for analysis.
zip_codes_df = pd.DataFrame(zip_codes_data)

# Display the sample ZIP Codes Table
# This section provides a preview and basic information about the data.
print("\nSample ZipCodes Table")  # Title for the report section
display(zip_codes_df.head())  # Display the first 5 rows of the DataFrame to preview the data.

# Show information about the DataFrame's structure and data types.
zip_codes_df.info()

# Print descriptive statistics for numerical columns, if any.
# Since this table is primarily categorical, statistics may not apply to all columns.
print("\nDescriptive Statistics for ZIP Codes Data:")
print(zip_codes_df.describe())


Zip Codes table created and saved to 'Zip_Codes.csv'

Sample ZipCodes Table


,ZIP_Code,Route_ID
0,85693,R2
1,83366,R76
2,82322,R46
3,32215,R29
4,12698,R93


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ZIP_Code  1000 non-null   object
 1   Route_ID  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB

Descriptive Statistics for ZIP Codes Data:
       ZIP_Code Route_ID
count      1000     1000
unique      999      100
top       64599      R61
freq          2       17


#3.Export and Download Data

##3.1 Export Dataframes

In [ ]:
# Export each DataFrame to a CSV file for external use and analysis.
# The CSV files can be imported into tools like Azure Data Studio or Excel.

# Save the Orders Table
orders_df.to_csv('Orders.csv', index=False)  # The index is excluded to avoid unnecessary columns.
print("Orders Table has been exported to 'Orders.csv'.")

# Save Product Table to CSV
# Save the Product DataFrame to a CSV file
products_df.to_csv("products_df.csv", index=False)
products_df

# Save the Distribution Centers Table
distribution_centers_df.to_csv('Distribution_Centers.csv', index=False)
print("Distribution Centers Table has been exported to 'Distribution_Centers.csv'.")

# Save the Carriers Table
carriers_df.to_csv('Carriers.csv', index=False)
print("Carriers Table has been exported to 'Carriers.csv'.")

# Save the Routes Details Table
routes_details_df.to_csv('Routes_Details.csv', index=False)
print("Routes Details Table has been exported to 'Routes_Details.csv'.")

# Save the Shipping Transactions Table
shipping_transactions_df.to_csv('Shipping_Transactions.csv', index=False)
print("Shipping Transactions Table has been exported to 'Shipping_Transactions.csv'.")

# Save the ZIP Codes Table
zip_codes_df.to_csv('Zip_Codes.csv', index=False)
print("ZIP Codes Table has been exported to 'Zip_Codes.csv'.")


Orders Table has been exported to 'Orders.csv'.
Distribution Centers Table has been exported to 'Distribution_Centers.csv'.
Carriers Table has been exported to 'Carriers.csv'.
Routes Details Table has been exported to 'Routes_Details.csv'.
Shipping Transactions Table has been exported to 'Shipping_Transactions.csv'.
ZIP Codes Table has been exported to 'Zip_Codes.csv'.


##3.2 Download CSV files

In [ ]:
# Enable downloading of the CSV files for local use.
# This is useful for quickly transferring data generated in the notebook to a local machine.

from google.colab import files  # Import the library to download files from Colab.

# Trigger download for each exported CSV file.
files.download('Orders.csv')  # Download Orders Table.
files.download('products_df.csv')
files.download('Distribution_Centers.csv')  # Download Distribution Centers Table.
files.download('Carriers.csv')  # Download Carriers Table.
files.download('Routes_Details.csv')  # Download Routes Details Table.
files.download('Shipping_Transactions.csv')  # Download Shipping Transactions Table.
files.download('Zip_Codes.csv') # Download Zip Codes Table.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>